In [206]:
RAW_MSC = '/home/tim/dv/bsv_inference/data_raw/temperature/UUEE.01.01.2014.31.03.2024.1.0.0.ru.utf8.00000000.csv'
RAW_NSK = '/home/tim/dv/bsv_inference/data_raw/temperature/UNNT.01.01.2014.31.03.2024.1.0.0.ru.utf8.00000000.csv'

In [207]:
import pandas as pd
import numpy as np

In [208]:
temp_moscow, temp_novosk = pd.read_csv(RAW_MSC, skip_blank_lines=True)[6:], pd.read_csv(RAW_NSK, skip_blank_lines=True)[6:]

In [209]:
temp_moscow.reset_index(drop=True, inplace=True)
temp_novosk.reset_index(drop=True, inplace=True)

In [210]:
temp_moscow.rename({temp_moscow.columns[0] : 'dt', temp_moscow.columns[1] : 'temp'},axis=1, errors="raise", inplace=True)
temp_novosk.rename({temp_novosk.columns[0] : 'dt', temp_novosk.columns[1] : 'temp'},axis=1, errors="raise", inplace=True)

In [211]:
temp_moscow = temp_moscow[::-1].reset_index(drop=True)
temp_novosk = temp_novosk[::-1].reset_index(drop=True)

In [212]:
temp_moscow = temp_moscow[pd.to_datetime(temp_moscow['dt']) >= pd.datetime(2020, 1, 1)].reset_index(drop=True)
temp_novosk = temp_novosk[pd.to_datetime(temp_novosk['dt']) >= pd.datetime(2020, 1, 1)].reset_index(drop=True)

/tmp/ipykernel_17960/2402446261.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  temp_moscow = temp_moscow[pd.to_datetime(temp_moscow['dt']) >= pd.datetime(2020, 1, 1)].reset_index(drop=True)
/tmp/ipykernel_17960/2402446261.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  temp_novosk = temp_novosk[pd.to_datetime(temp_novosk['dt']) >= pd.datetime(2020, 1, 1)].reset_index(drop=True)


In [217]:
def temp_interpolate(df, hour):
    start_date = pd.to_datetime("2020-01-01")
    end_date = pd.to_datetime("2024-03-31")
    df['dt'] = pd.to_datetime(df['dt'])
    date_range_df = pd.DataFrame(pd.date_range(start=start_date, end=end_date, freq='D')+ pd.Timedelta(hours=hour), columns=['date_range'])
    missing_dates_df = pd.merge(date_range_df, df, left_on='date_range', right_on='dt', how='left', indicator=True)
    missing_dates = missing_dates_df[missing_dates_df['_merge'] == 'left_only']['date_range']

    blank_rows = pd.DataFrame(columns=['dt'])
    blank_rows['dt'] = missing_dates

    result_df = pd.concat([df, blank_rows]).sort_values('dt').reset_index(drop=True)
    result_df['temp'] = result_df['temp'].interpolate(method='linear')

    return result_df

In [222]:
def to_farenheit(temp_c):
    return float(temp_c) * 1.8 + 32 + 48

def get_temp(df, hour):
    df['dt'] = pd.to_datetime(df['dt'], format='%d.%m.%Y %H:%M')

    df = df[(df['dt'].dt.minute == 0)]
    df = df[df['dt'].dt.hour == hour]
    df['temp'] = df['temp'].apply(to_farenheit)
    df = temp_interpolate(df, hour)
    # df['dt'] = pd.to_datetime(df['dt'], format='%d.%m.%Y %H:%M')
    # df = df[(df['dt'].dt.minute == 0)]
    # df = df[df['dt'].dt.hour == hour]
    df.to_csv(f'/home/tim/dv/bsv_inference/data_raw/temperature_raw/zone_2/zone_2_hour_{hour}.csv', index=False)
    return

In [223]:
for hour in range(24):
    get_temp(temp_novosk, hour=hour)

In [80]:
temp_moscow['dt'] = pd.to_datetime(temp_moscow['dt'], format='%d.%m.%Y %H:%M')
temp_novosk['dt'] = pd.to_datetime(temp_novosk['dt'], format='%d.%m.%Y %H:%M')

In [83]:
temp_moscow['temp'] = temp_moscow['temp'].apply(to_farenheit)
temp_novosk['temp'] = temp_novosk['temp'].apply(to_farenheit)

for_min_1 = temp_moscow
for_min_1['dt'] = for_min_1['dt'].dt.date

for_min_2 = temp_novosk
for_min_2['dt'] = for_min_2['dt'].dt.date

for_min_1 = for_min_1.groupby('dt')['temp'].min().reset_index(drop=True)
for_min_2 = for_min_2.groupby('dt')['temp'].min().reset_index(drop=True)

for_min_1.to_csv('/home/tim/dv/bsv_inference/data_raw/temperature/min_zone1.csv', index=False)
for_min_2.to_csv('/home/tim/dv/bsv_inference/data_raw/temperature/min_zone2.csv', index=False)